In [1]:
import os
os.environ["HF_HUB_CACHE"] = "/NS/formal-grammar-and-memorization/nobackup/shared/huggingface_cache/hub"
os.environ["HF_DATASETS_CACHE"] = "/NS/formal-grammar-and-memorization/nobackup/shared/huggingface_cache/datasets"

In [2]:
# from datasets import load_dataset
# ds = load_dataset("nyu-mll/multi_nli")

import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation_matched': 'data/validation_matched-00000-of-00001.parquet', 'validation_mismatched': 'data/validation_mismatched-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/nyu-mll/multi_nli/" + splits["train"])
df_val_matched = pd.read_parquet("hf://datasets/nyu-mll/multi_nli/" + splits["validation_matched"])
df_val_mismatched = pd.read_parquet("hf://datasets/nyu-mll/multi_nli/" + splits["validation_mismatched"])

In [3]:
label_map = {
    1: "neutral",
    0: "entailment",
    2: "contradiction",
}

In [4]:
import random
def process_columns(df):
    random.seed(5)
    df['label_flipped'] = df['label'].apply(
        lambda x: random.choice([label for label in [0, 1, 2] if label != x])
    )
    assert (df['label_flipped'] != df['label']).all()
    df['label_str'] = df['label'].map(label_map)
    df['label_flipped_str'] = df['label_flipped'].map(label_map)
    df['in-language'] = 'Premise: ' + df["premise"] + '\nHyopthesis: ' + df["hypothesis"] + '\nLabel: ' + df["label_str"]
    df['out-language'] = 'Premise: ' + df["premise"] + '\nHyopthesis: ' + df["hypothesis"] + '\nLabel: ' + df["label_flipped_str"]
    return df

df_train = process_columns(df_train)
df_val_matched = process_columns(df_val_matched)
df_val_mismatched = process_columns(df_val_mismatched)

In [5]:
instruction = """Provide a classification label for the pair, indicating the relationship between the premise and hypothesis:
- entailment : The hypothesis logically follows from the premise.
- neutral : The hypothesis is neither entailed nor contradicted by the premise.
- contradiction : The hypothesis contradicts the premise.


"""

In [21]:
print(df_train.iloc[11]['in-language'])

Premise: Fun for adults and children.
Hyopthesis: Fun for only children.
Label: contradiction


In [22]:
print(df_train.iloc[11]['out-language'])

Premise: Fun for adults and children.
Hyopthesis: Fun for only children.
Label: neutral


In [8]:
print(df_val_mismatched.iloc[2]['in-language'])

Premise:  We serve a classic Tuscan meal that includes a Florentine terrine made with dick and chicken livers.
Hyopthesis: We serve a meal of Florentine terrine.
Label: entailment


In [9]:
df_train['genre'].unique()

array(['government', 'telephone', 'fiction', 'travel', 'slate'],
      dtype=object)

In [10]:
df_val_matched.groupby('genre').count()

,promptID,pairID,premise,premise_binary_parse,premise_parse,hypothesis,hypothesis_binary_parse,hypothesis_parse,label,label_flipped,label_str,label_flipped_str,in-language,out-language
genre,,,,,,,,,,,,,,
fiction,1973,1973,1973,1973,1973,1973,1973,1973,1973,1973,1973,1973,1973,1973
government,1945,1945,1945,1945,1945,1945,1945,1945,1945,1945,1945,1945,1945,1945
slate,1955,1955,1955,1955,1955,1955,1955,1955,1955,1955,1955,1955,1955,1955
telephone,1966,1966,1966,1966,1966,1966,1966,1966,1966,1966,1966,1966,1966,1966
travel,1976,1976,1976,1976,1976,1976,1976,1976,1976,1976,1976,1976,1976,1976


In [11]:
df_val_matched['genre'].unique()

array(['slate', 'government', 'telephone', 'travel', 'fiction'],
      dtype=object)

In [12]:
# select one genre only
df_train = df_train[df_train['genre'] == 'government']
df_val_matched = df_val_matched[df_val_matched['genre'] == 'government']

df_train.shape, df_val_matched.shape, df_val_mismatched.shape

((77350, 15), (1945, 15), (9832, 15))

In [13]:
df_val_mismatched['genre'].unique()

array(['letters', 'verbatim', 'facetoface', 'oup', 'nineeleven'],
      dtype=object)

In [14]:
df_train['in-language_len'] = df_train['in-language'].apply(lambda x: len(x.strip()))
df_train['out-language_len'] = df_train['out-language'].apply(lambda x: len(x.strip()))
df_val_matched['in-language_len'] = df_val_matched['in-language'].apply(lambda x: len(x.strip()))
df_val_matched['out-language_len'] = df_val_matched['out-language'].apply(lambda x: len(x.strip()))
df_val_mismatched['in-language_len'] = df_val_mismatched['in-language'].apply(lambda x: len(x.strip()))
df_val_mismatched['out-language_len'] = df_val_mismatched['out-language'].apply(lambda x: len(x.strip()))

In [15]:
df_train[df_train['in-language_len'] == 0].shape, df_train[df_train['out-language_len'] == 0].shape, df_val_matched[df_val_matched['in-language_len'] == 0].shape, df_val_matched[df_val_matched['out-language_len'] == 0].shape, df_val_mismatched[df_val_mismatched['in-language_len'] == 0].shape, df_val_mismatched[df_val_mismatched['out-language_len'] == 0].shape

((0, 17), (0, 17), (0, 17), (0, 17), (0, 17), (0, 17))

In [16]:
fa

NameError: name 'fa' is not defined

In [17]:
mnli_dataset_in_distribution_dict = {
    "train_sequences": df_train['in-language'].tolist(),
    "test_sequences": df_val_matched['in-language'].tolist(),
    "non_grammatical_test_sequences_edit_distance_1": df_val_matched['out-language'].tolist(),
}


mnli_dataset_out_distribution_dict = {
    "train_sequences": df_train['in-language'].tolist(),
    "test_sequences": df_val_mismatched['in-language'].tolist(),
    "non_grammatical_test_sequences_edit_distance_1": df_val_mismatched['out-language'].tolist(),
}

In [18]:
print(mnli_dataset_in_distribution_dict["test_sequences"][0])

Premise: This site includes a list of all award winners and a searchable database of Government Executive articles.
Hyopthesis: The Government Executive articles housed on the website are not able to be searched.
Label: contradiction


In [19]:
print(mnli_dataset_in_distribution_dict["non_grammatical_test_sequences_edit_distance_1"][0])

Premise: This site includes a list of all award winners and a searchable database of Government Executive articles.
Hyopthesis: The Government Executive articles housed on the website are not able to be searched.
Label: neutral


In [20]:
print(mnli_dataset_out_distribution_dict["test_sequences"][0])

Premise: Your contribution helped make it possible for us to provide our students with a quality education.
Hyopthesis: Your contributions were of no help with our students' education.
Label: contradiction


In [21]:
print(mnli_dataset_out_distribution_dict["non_grammatical_test_sequences_edit_distance_1"][0])

Premise: Your contribution helped make it possible for us to provide our students with a quality education.
Hyopthesis: Your contributions were of no help with our students' education.
Label: neutral


In [22]:
filename_in_distribution = "../data/mnli_dataset_in_distribution/sequences_w_edit_distance_mnli_dataset_in_distribution_10000_5.pkl"
filename_out_distribution = "../data/mnli_dataset_out_distribution/sequences_w_edit_distance_mnli_dataset_out_distribution_10000_5.pkl"

os.system(f"mkdir -p ../data/mnli_dataset_in_distribution")
os.system(f"mkdir -p ../data/mnli_dataset_out_distribution")


filename_in_distribution_instruction = "../data/mnli_dataset_in_distribution/instruction_mnli_dataset_in_distribution.pkl"
filename_out_distribution_instruction = "../data/mnli_dataset_out_distribution/instruction_mnli_dataset_out_distribution.pkl"


import pickle

with open(filename_in_distribution, 'wb') as f:
    pickle.dump(mnli_dataset_in_distribution_dict, f)

with open(filename_out_distribution, 'wb') as f:
    pickle.dump(mnli_dataset_out_distribution_dict, f)

with open(filename_in_distribution_instruction, 'wb') as f:
    pickle.dump(instruction, f)

with open(filename_out_distribution_instruction, 'wb') as f:
    pickle.dump(instruction, f)
